word文档 问题：==》修改
-  斜率是否这样计算？这个推理不应该是对的： 即每一个单位(上升或下降的值为a) 的利率参数变动对应客户数量的变动为.特别是$i_{\min }$不为零情况

<font size = 4 color ="blue">

x轴=利率变化 :  $\frac{i_{\max }}{a}$ - $\frac{i_{\min }}{a}$   

y轴=客户数变化 :  N - 0 = N

y/x利率:   $\frac{N * a} {(i_{\max } - i_{\min })}$

如果 $i_{\min }$ = 0 ,$\frac{N * a} {i_{\max }}$
</font>


- 模型

N/B


# 1.需求
多名学生分别扮演多个银行，同时对各自的利率和手续费等影响因素进
行调节，在每个学生分别调节完后录入仿真试验系统中。通过特定的模型
算法产生对抗，根据每个学生操作的银行的影响因素的大小进行反馈并给
出客户数量和总的资金流。该项仿真试验模型的创新点为运用统计和计量
经济学知识，结合金融的EGARCH 模型思想已经宏观经济学中的动态均
衡思想。<br>
<b>输入：</b>利率(α的倍数）和手续费（b的倍数）等影响成本的因<br>
<b>输出：</b>客户数和资金流。<br>
<b>算法：</b>EGARCH <br>
<b>场景:</b>
-	单一银行的单一要素对客户和资金流的分布的具体影响；
-	单一银行的多种要素同时变动，多种要素对客户和资金流的分布的混合影响；
-	多个银行同时存在时，多个银行分别调整单一要素同时作用于客户和资金流，客户和资金流会被如何影响；
-	多个银行同时存在时，并且多种要素同时变动时，多个银行分别调整多种要素并同时作用于客户和资金流，客户和资金流会被如何影响；


In [2]:
import ipywidgets as widgets
import numpy as np

# 2.模型
<b>银行</b>：市场上存在B家银行，银行的利率参数$\mathrm{r}_{\mathrm{m}, 1}$（可以在一个区间内调节：$\dot{I} \in\left[\mathrm{i}_{\mathrm{min}}, \mathrm{i}_{\mathrm{max}}\right]$。）和手续成本参数r_(m,2)(在一个区间内调节：$c \in\left[0, c_{\max }\right]$)
<br>
<b>客户</b>：市场上存在N个客户（商户），每个客户带来的资金量为固定值M, 则该虚拟仿真市场上存在的总资金量为N 乘以M；同时定义在最初时刻（即试验最开始环节），每家银行分到的客户和资金量为均等的，其对应的值为NM/B ；

<b> 客户量$\mathrm{N}_{\mathrm{m}}$ =N；资金流$M_{m}=N_{m} M$</b>
<font color ="blue" size =4>
$\begin{aligned} N_{m}^{*}=N / B+p_{m, 1}(B-1)\left(\right.\text {change} \left._{m, 1}\right)-& \sum_{k \neq m}^{B} p_{k, 1} \operatorname{change}_{k, 1} +p_{m, 2}(B-1)\left(\right.\text {change} \left._{m, 2}\right)-\sum_{k \neq m}^{B} p_{k, 2} \operatorname{change}_{k, 2} \end{aligned}$
 
 $N_{m}^{*}=N / B+ B* \left(p_{m, 1}\right.$ change $\left._{m, 1}\right)-\sum_{k }^{B} p_{k, 1}$ change $_{k, 1}+ B *\left(p_{m, 2}\right.$ change $\left._{m, 2}\right)-\sum_{k }^{B} p_{k, 2}$ change $_{k, 2}$
 
或 ：<br>
$\begin{aligned} N_{m}^{*}=N / B+\sum_{k=1 }^{B}\left(p_{m, 1}\right.\text {change} \left._{m, 1}- p_{k, 1} \text {change}_{k, 1}\right) +\sum_{k=1 }^{B}\left(p_{m, 2}\right.\text {change} \left._{m, 2}- p_{k, 2} \text {change}_{k, 2}\right)\end{aligned}$

$\begin{aligned} N_{m}^{*}=N / B+ factor1 +factor2\end{aligned}$
  
</font>
已证明 ：市场永远均衡，客户数和等于总客户数
    
# 3.老师参数
global_paras

In [3]:
# 市场上有B家银行
B = 5
# 市场上有100个客户
N = 100
# 每个客户的资金量，单位：元
M =10000
# 市场上的利率区间,存款利率
i_min = 0
i_max = 1
# 市场上的手续费区间区间
c_min = 0
c_max = 0.5
# 市场上利率调节，幅度步长 a%
a = 0.25
# 市场上手续费条件，幅度步长b%
b = 0.001
# 利率影响因素权重
w1 = 0.3

In [4]:
bank_list = ["bank_"+str(i+1) for i in range(B)]
bank_list

['bank_1', 'bank_2', 'bank_3', 'bank_4', 'bank_5']

<font color ="blue" size =4>定义模型 </font>

In [5]:
class bank():
    def __init__(self,B,N,M,i_min,i_max,c_min,c_max,a,b,w1):
        self.B = B
        self.N = N
        self.M = M
        self.i_min = i_min
        self.i_max = i_max
        self.c_min = c_min
        self.c_max = c_max        
        self.a = a
        self.b = b
        self.w1 = w1
                
       # 交易手续费影响权重
        self.w2 = 1 - w1 
        
    def  cal_n(self,p1,p2,r1,r2) :  
        #斜率
        perc1 =  self.N /(self.i_max - self.i_min)
        perc2 =  -self.N /(self.c_max - self.c_min)  #附利率 ,可以带入等式中使用
        
        change1 = [self.w1 * x * perc1 for x in r1]  # x - 0
        change2 = [self.w2 * x * perc2 for x in r2]

        f1 =  np.array(p1) * np.array(change1)
        factor1 = self.B*f1 - sum(f1)

        f2 =  np.array(p2) * np.array(change2)
        factor2 = self.B * f2 - sum(f2)

        N_cust = [int(x) for x in [self.N/self.B] + factor1 + factor2]  # 客户数
        N_money = [x * self.M for x in N_cust]                       # 资金量
        print ("客户数     资金量")
        return N_cust ,N_money
my_scen = bank(B,N,M,i_min,i_max,c_min,c_max ,a,b,w1) #场景

# 4.初始状态
ini_set

In [6]:
p1 = [0]*B 
p2 = [0]*B
r1 = [0]*B  #利率调整幅度
r2 = [0]*B  #手续费调整幅度
p2

[0, 0, 0, 0, 0]

# 5.学生设置
student_paras
## 场景选择

### 5.1 单一银行的单一要素对客户和资金流的分布的具体影响；

In [22]:
one_one = widgets.HBox([
 widgets.Dropdown(
    options=bank_list,
    value='bank_1',
    description='银行:',
    disabled=False,
),
widgets.RadioButtons(
    options=['利率', '手续费'],
#     value='pineapple',
    description='调节指标:',
    disabled=False
),
widgets.RadioButtons(
    options=["上升", "下降"],
#     value='pineapple',
    description='调节方向:',
    disabled=False
),
widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1,
    step=0.25,  # 是关于指标类型的变量
    description='调整幅度(%):',
    disabled=False
)    
    
])
one_one

In [8]:
# 初始状态
p1 = [0]*B 
p2 = [0]*B
r1 = [0]*B  #利率调整幅度
r2 = [0]*B  #手续费调整幅度

i = list(one_one.children[0].options).index(one_one.children[0].value) # 第i家银行指标有变动
if one_one.children[1].value == "利率" :
    p1[i] = 1 if one_one.children[2].value == "上升" else -1
    r1[i] = one_one.children[3].value
elif   one_one.children[1].value == "手续费" :  
    p2[i] = 1 if one_one.children[2].value == "上升" else -1
    r2[i] = one_one.children[3].value
print("利率p1 : {},  手续费p2 : {} , 利率r1 : {} ,  手续费r2 : {}".format(p1,p2,r1,r2))

利率p1 : [1, 0, 0, 0, 0],  手续费p2 : [0, 0, 0, 0, 0] , 利率r1 : [0.0, 0, 0, 0, 0] ,  手续费r2 : [0, 0, 0, 0, 0]


<font color ="red" size =4>调用模型 </font>

In [9]:
my_scen.cal_n(p1,p2,r1,r2)

客户数     资金量


([20, 20, 20, 20, 20], [200000, 200000, 200000, 200000, 200000])

### 5.2 单一银行的多种要素同时变动，多种要素对客户和资金流的分布的混合影响；

In [23]:
one_mul = widgets.VBox([ widgets.Dropdown(
    options=bank_list,
    value= "bank_1",
    description='银行:',
    disabled=False,
),widgets.HBox([
widgets.Label(value="  调解利率") ,
widgets.RadioButtons(
    options=["上升", "下降"],
#     value='pineapple',
    description='调节方向:',
    disabled=False
),
widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1,
    step=0.25,  # 是关于指标类型的变量
    description='调整幅度(%):',
    disabled=False
)    
    
]),widgets.HBox([
widgets.Label(value="  调解手续费") ,
widgets.RadioButtons(
    options=["上升", "下降"],
#     value='pineapple',
    description='调节方向:',
    disabled=False
),
widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1,
    step=0.001,  # 是关于指标类型的变量
    description='调整幅度(%):',
    disabled=False
)    
    
])
                       ])
one_mul

In [11]:
# 初始状态
p1 = [0]*B 
p2 = [0]*B
r1 = [0]*B  #利率调整幅度
r2 = [0]*B  #手续费调整幅度

# 读取修改数据
i = list(one_mul.children[0].options).index(one_mul.children[0].value) # 第i家银行指标有变动

p1[i] = 1 if one_mul.children[1].children[1].value == "上升" else -1
r1[i] = one_mul.children[1].children[2].value

p2[i] = 1 if one_mul.children[2].children[1].value == "上升" else -1
r2[i] = one_mul.children[2].children[2].value
print("利率 p1 : {},  手续费 p2 : {} , 利率 r1 : {} ,  手续费 r2 : {}".format(p1,p2,r1,r2))

利率 p1 : [1, 0, 0, 0, 0],  手续费 p2 : [1, 0, 0, 0, 0] , 利率 r1 : [0.0, 0, 0, 0, 0] ,  手续费 r2 : [0.0, 0, 0, 0, 0]


In [12]:
# 调用模型
my_scen.cal_n(p1,p2,r1,r2)

客户数     资金量


([20, 20, 20, 20, 20], [200000, 200000, 200000, 200000, 200000])

### 5.3 多个银行同时存在时，多个银行分别调整单一要素同时作用于客户和资金流，客户和资金流会被如何影响；

In [13]:
list_box = [widgets.RadioButtons(
    options=['利率', '手续费'],
#     value='pineapple',
    description='调节指标:',
    disabled=False
)]
for i in range(B):
    one_bank = widgets.HBox([
                 widgets.Dropdown(
                    options=['A', 'B', 'C'],
                    value='B',
                    description='银行:',
                    disabled=False,
                ),
                widgets.RadioButtons(
                    options=["上升", "下降"],
                #     value='pineapple',
                    description='调节方向:',
                    disabled=False
                ),
                widgets.BoundedFloatText(
                    value=0,
                    min=0,
                    max=1,
                    step=0.25,  # 是关于指标类型的变量
                    description='调整幅度(%):',
                    disabled=False
                )    

                ])
    list_box.append(one_bank)
list_box.append(widgets.Button(
    description='确定',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
) )
mul_one = widgets.VBox(list_box)
mul_one

In [14]:
# 初始状态
p1 = [0]*B 
p2 = [0]*B
r1 = [0]*B  #利率调整幅度
r2 = [0]*B  #手续费调整幅度

# 读取修改数据

if mul_one.children[0].value  == "利率":
    for i in range(B) :
        p1[i] = 1 if mul_one.children[i+1].children[1].value == "上升" else -1
        r1[i] = mul_one.children[i+1].children[2].value        
elif mul_one.children[0].value  == "手续费":
    for i in range(B) :
        p2[i] = 1 if mul_one.children[i+1].children[1].value == "上升" else -1
        r2[i] = mul_one.children[i+1].children[2].value
print("利率 p1 : {},  手续费 p2 : {} , 利率 r1 : {} ,  手续费 r2 : {}".format(p1,p2,r1,r2))

利率 p1 : [1, 1, 1, 1, 1],  手续费 p2 : [0, 0, 0, 0, 0] , 利率 r1 : [0.0, 0.0, 0.0, 0.0, 0.0] ,  手续费 r2 : [0, 0, 0, 0, 0]


In [15]:
# 调用模型
my_scen.cal_n(p1,p2,r1,r2)

客户数     资金量


([20, 20, 20, 20, 20], [200000, 200000, 200000, 200000, 200000])

### 5.4 多个银行同时存在时，并且多种要素同时变动时，多个银行分别调整多种要素并同时作用于客户和资金流，客户和资金流会被如何影响；

In [16]:
# 和B有关
list_box = []
for i in range(B) :
    
        one_box = widgets.HBox([ widgets.Dropdown(
            options=bank_list,
            value='bank_1',
            description='银行:',
            disabled=False,
        ),widgets.HBox([

        widgets.RadioButtons(
            options=["上升", "下降"],
        #     value='pineapple',
            description='调节利率:',
            disabled=False
        ),
        widgets.BoundedFloatText(
            value=0,
            min=0,
            max=1,
            step=0.25,  # 是关于指标类型的变量
            description='幅度(%):',
            disabled=False
        )    

        ]),
         widgets.HBox([


         widgets.RadioButtons(
            options=["上升", "下降"],
        #     value='pineapple',
            description='调节手续费:',
            disabled=False
        ),
        widgets.BoundedFloatText(
            value=0,
            min=0,
            max=1,
            step=0.001,
            description='幅度(%):',
            disabled=False
        )    

        ]),


                                ])
        list_box.append(one_box)
mul_mul = widgets.VBox(list_box)
mul_mul

In [17]:
mul_mul.children[0].children[2].children[1].value

0.0

In [18]:
# 初始状态
p1 = [0]*B 
p2 = [0]*B
r1 = [0]*B  #利率调整幅度
r2 = [0]*B  #手续费调整幅度

# 读取修改数据
for i in range(B):
    p1[i] = 1 if mul_mul.children[i].children[1].children[0].value == "上升" else -1
    r1[i] = mul_mul.children[i].children[1].children[1].value
    p2[i] = 1 if mul_mul.children[i].children[2].children[0].value == "上升" else -1
    r2[i] = mul_mul.children[i].children[2].children[1].value
    
print("利率 p1 : {},  手续费 p2 : {} , 利率 r1 : {} ,  手续费 r2 : {}".format(p1,p2,r1,r2))    

利率 p1 : [1, 1, 1, 1, 1],  手续费 p2 : [1, 1, 1, 1, 1] , 利率 r1 : [0.0, 0.0, 0.0, 0.0, 0.0] ,  手续费 r2 : [0.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
# 调用模型
my_scen.cal_n(p1,p2,r1,r2)

客户数     资金量


([20, 20, 20, 20, 20], [200000, 200000, 200000, 200000, 200000])